### Imports

In [1]:
import json
import time
import pandas as pd
from py2neo import Graph, Node, Relationship
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

### Connect to graph

In [2]:
graph = Graph("bolt://neo4j-magone:7687", auth=('neo4j','myneo'))
top_42 = ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']

print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

Connected to graph database with 278,432,359 nodes and 1,844,501,832 relationships!


### Build dataset

In [ ]:
years_to_use = 1
start_year = 2016
end_year = 2019

print("Getting dataset...", end=" ")
cites_str = ',\n    '.join(['CASE WHEN {} < q.year THEN NULL ELSE SIZE((q)<-[:CITES]-(:Quanta {{year: {}}})) END as c{}'.format(
    yr, yr, yr) for yr in range(start_year, end_year+1)])
tspr_str = ',\n    '.join(['q.tspr{} as tspr{}'.format(
    yr, yr) for yr in range(start_year, end_year+1)])
query = """
MATCH (q:Quanta)
WHERE 
    q.year>={} AND q.year <= {} AND q.venue IN """+str(top_42)+"""
RETURN
    q.year as year,
    q.title as title,
    q.id as id,
    q.venue as venue,
    {},
    {}
"""

query = query.format(start_year, end_year-years_to_use, tspr_str, cites_str)
print(query)
query_start_time = time.time()
df = graph.run(query).to_data_frame()
print("Done ({:.2f} minutes).".format((time.time()-query_start_time)/60))

df.to_pickle('/tmp/data/result/top_42_2016_2018.pkl')


Getting dataset... 
MATCH (q:Quanta)
WHERE 
    q.year>=2016 AND q.year <= 2018 AND q.venue IN ['Angewandte Chemie', 'Blood', 'Cancer Cell', 'Cancer Discovery', 'Cancer Research', 'Cell', 'Cell Host & Microbe', 'Cell Metabolism', 'Cell Stem Cell', 'Chemistry & Biology', 'The EMBO Journal', 'Genes & Development', 'Immunity', 'Journal of Neurology', 'Journal of the American Chemical Society', 'JAMA', 'Journal of Biological Chemistry', 'Journal of Cell Biology', 'Journal of Clinical Investigation', 'Journal of Experimental Medicine', 'Journal of Medicinal Chemistry', 'The Lancet', 'Nature Cell Biology', 'Nature Chemical Biology', 'Nature Chemistry', 'Nature Medicine', 'Nature Methods', 'Nature', 'Nature Biotechnology', 'The New England Journal of Medicine', 'Neuron', 'Nature Genetics', 'Nature Immunology', 'Nature Neuroscience', 'Nature Structural & Molecular Biology', 'PLOS Biology', 'PLOS Genetics', 'PLOS Pathogens', 'Proceedings of the National Academy of Sciences of the United States 

In [ ]:
df_new = df[['year','title','id']]
for i in range(years_to_use+1):
    df_new['c{}'.format(i)] = df.apply(lambda row: row['c{}'.format(row['year']+i)], axis=1)
    df_new['p{}'.format(i)] = df.apply(lambda row: row['tspr{}'.format(row['year']+i)], axis=1)


# Add in community features


In [ ]:
import glob
feature_paths = ["/tmp/data/result/FeatureExtractionResults/EarlyAdopters/"]

for feature_path in feature_paths:
    all_files = sorted(glob.glob(feature_path +"*.csv"), reverse=True)
    feature_vec_chunks = []
    for file in all_files:
        feature_vec_chunks.append(pd.read_csv(file))
        
    total_feature = pd.concat(feature_vec_chunks)
#     print(sorted(total_feature.title))
    df_features = df_new.merge(pd.concat(feature_vec_chunks), on='title')

# ONLY USES FEATURES WITH FULL SERIES
df_features = df_features.dropna()
df_features

In [ ]:
def balanced_subsample(x,y,subsample_size=1.0):
    class_xs = []
    min_elems = None
    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]
    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)
    xs = []
    ys = []
    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            this_xs = this_xs.reindex(np.random.RandomState(seed=42).permutation(this_xs.index))
        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)
        xs.append(x_)
        ys.append(y_)

    xs = pd.concat(xs)
    ys = pd.Series(data=np.concatenate(ys),name='target')
    return xs,ys

# Feature extraction + Class Labeling
# Train + Test Split
## Balanced Subsampling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

cs = ['c{}'.format(x) for x in range(years_to_use+1)]
ps = ['p{}'.format(x) for x in range(years_to_use+1)]
ea = ['early_adopters_{}'.format(x) for x in range(1,years_to_use+1)]
cs.extend(ps)
cs.extend(ea)
X = df_features[cs]
y = df_features['p{}'.format(years_to_use)] >= df_features['p{}'.format(years_to_use)].quantile(0.95)


to_keep = []
num_signals = 1
for i in range(years_to_use+1):
    
    if i == num_signals:
        break
#     to_keep.append('c{}'.format(i))
    to_keep.append('p{}'.format(i))
    pass

for i in range(1, years_to_use+1):
    if i-1 == num_signals:
        break
#     to_keep.append('early_adopters_{}'.format(i))

y = label_binarize(y, classes=[True,False])
n_classes = y.shape[1]


X_restricted = X[to_keep]
X_train, X_test, y_train, y_test = train_test_split(X_restricted, y, test_size=0.33, random_state=42)
X_bal_train, y_bal_train = balanced_subsample(X_train,y_train)

In [ ]:
!pip install scikit-learn
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, recall_score, accuracy_score


sensitivity = make_scorer(recall_score, pos_label = 1)
specificity = make_scorer(recall_score, pos_label = 0)

scoring = {'acc': 'accuracy',
           'sensitivity': sensitivity,
           'specificity': specificity}
# clf = DecisionTreeClassifier(random_state=0)
clf = RandomForestClassifier(random_state=0)
cv_results = cross_validate(clf, X_bal_train, y_bal_train, scoring=scoring,
                         cv=5, return_train_score=True, return_estimator=True)


best_est_index = np.argmax(cv_results['test_acc'])
best_estimator = cv_results['estimator'][best_est_index]

sensitivity(best_estimator, X_test, y_test), specificity(best_estimator, X_test, y_test)


In [ ]:
from sklearn.metrics import roc_curve, auc

y_score = best_estimator.predict_proba(X_test)



# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()


roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, 1]) # HACK MUST FIX THIS IMMEDIATELY, PROB(CLASS = 1) IS 1ST INDEX IN Y_SCORE
    roc_auc[i] = auc(fpr[i], tpr[i])


plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic curve')
plt.legend(loc="lower right")
plt.show()

In [3]:

blah = pd.read_pickle('/tmp/data/result/FeatureExtractionResults/top_42_2000_2015.pkl')

In [4]:
blah

,c2000,c2001,c2002,c2003,c2004,c2005,c2006,c2007,c2008,c2009,...,tspr2011,tspr2012,tspr2013,tspr2014,tspr2015,tspr2016,tspr2017,tspr2018,venue,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.028478,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2013
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000939,0.215108,0.253897,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2011
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003045,0.085605,0.175725,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2010
3,NaN,NaN,NaN,NaN,NaN,NaN,5.0,10.0,7.0,6.0,...,0.453946,0.456230,0.486391,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2006
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.279245,0.541166,0.735157,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2011
5,NaN,0.0,7.0,13.0,13.0,12.0,14.0,9.0,6.0,6.0,...,0.547719,0.540082,0.523901,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2001
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.019894,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2013
7,NaN,NaN,NaN,NaN,NaN,2.0,2.0,0.0,2.0,1.0,...,0.004548,0.000097,0.011795,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2005
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2014
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.019894,NaN,NaN,NaN,NaN,NaN,Nature Structural & Molecular Biology,2013
